In [10]:
import numpy as np
import scipy.io
import os

In [29]:
inp_file = "/home/kristjan/work/sts_cp2k/restart_files/RESTART_orig"

inpf = open(inp_file, 'r')

_h, natom, nspin, nao, nset_max, nshell_max, _h = np.fromfile(inpf, dtype=np.int32, count=7)

print(np.fromfile(inpf, dtype=np.int32, count=7))


[16  1  1  1  1 16 16]


In [128]:
mu = -8
kt = 8.6173303e-5*300.0
evals = np.linspace(-10, 0.0, 4)
occs = 2.0/(np.exp((evals-mu)/(kt))+1)
occs

array([  2.00000000e+000,   7.98009643e-023,   8.02515627e-079,
         8.07047054e-135])

In [139]:
def calc_fermi(evals, occs):
    # Find closest value to 1.0 (half occupancy)
    i1 = np.argsort(np.abs(occs-1.0))[0]
    eps = 1e-9
    
    # if no smearing, find last occupied energy
    if occs[i1] > 2.0-eps or occs[i1] < eps:
        first_empty = next(x[0] for x in enumerate(occs) if x[1] < 1.0)
        return evals[first_empty - 1]
    # if the closest one is exactly half-filled, we have the Fermi level!
    if np.abs(occs[i1] - 1.0) < eps:
        return evals[i1]
    
    # second closest value
    prev_occ = 2.0 - occs[i1-1]
    next_occ = occs[i1+1]
    
    if prev_occ > next_occ:
        i2 = i1 - 1
    else:
        i2 = i1 + 1
    
    # with smearing, calculate the chemical potential
    n1 = occs[i1]/2
    n2 = occs[i2]/2
    e1 = evals[i1]
    e2 = evals[i2]
    # log(1/n - 1) = log(1-n) - log(n)
    # np.log1p(x) = np.log(1 + x)
    log_div = (np.log1p(-n1) - np.log(n1))/(np.log1p(-n2) - np.log(n2))
    return (e1 - e2*log_div)/(1 - log_div)
    

In [140]:
calc_fermi(evals, occs)

-10.0

In [127]:
# RESTART to ASCII

inp_file = "/home/kristjan/work/sts_cp2k/restart_files/RESTART_orig"
out_file = "/home/kristjan/work/sts_cp2k/restart_files/RESTART.ascii"

inpf = scipy.io.FortranFile(inp_file, 'r')
outf = open(out_file, 'w')

natom, nspin, nao, nset_max, nshell_max = inpf.read_ints()
outf.write("%d %d %d %d %d\n" % (natom, nspin, nao, nset_max, nshell_max))

# natom - number of atoms
# nspin - number of spins
# nao - number of atomic orbitals
# nset_max - maximum number of sets in the basis set
#           (e.g. if one atom's basis set contains 3 sets and every other 
#           atom's contains 1, then this value will still be 3)
# nshell_max - maximum number of shells in each set

nset_info = inpf.read_ints() # number of sets in the basis set for each atom
outf.write(' '.join(map(str, nset_info))+"\n")

nshell_info = inpf.read_ints() # number of shells in each of the sets
outf.write(' '.join(map(str, nshell_info))+"\n")

nso_info = inpf.read_ints() # number of orbitals in each shell
outf.write(' '.join(map(str, nso_info))+"\n")

for ispin in range(nspin):
    nmo, homo, lfomo, nelectron = inpf.read_ints()
    outf.write("%d %d %d %d\n" % (nmo, homo, lfomo, nelectron))
    
    # nmo - number of molecular orbitals
    # homo - index of the HOMO
    # lfomo - ???
    # nelectron - number of electrons
    
    # list containing all eigenvalues and occupancies of the molecular orbitals
    evals_occs = inpf.read_reals() 
    outf.write(' '.join(map(str, evals_occs))+"\n")
    
    for imo in range(nmo):
        # list of coefficients corresponding to each shell-orbital
        # NB: the coefficients assume spherical basis set representations!
        coefs = inpf.read_reals()
        outf.write(' '.join(map(str, coefs))+"\n")
    
inpf.close()
outf.close()

In [5]:
# ASCII to RESTART

inp_file = "/home/kristjan/work/sts_cp2k/restart_files/RESTART.ascii"
out_file = "/home/kristjan/work/sts_cp2k/restart_files/RESTART"

inpf = open(inp_file, 'r')
outf = scipy.io.FortranFile(out_file, 'w')

line1 = [int(d) for d in inpf.readline().split(' ')]
outf.write_record(np.array(line1, dtype=np.intc))
natom, nspin, nao, nset_max, nshell_max = line1


nset_info = [int(d) for d in inpf.readline().split(' ')]
outf.write_record(np.array(nset_info, dtype=np.intc))

nshell_info = [int(d) for d in inpf.readline().split(' ')]
outf.write_record(np.array(nshell_info, dtype=np.intc))

nso_info = [int(d) for d in inpf.readline().split(' ')]
outf.write_record(np.array(nso_info, dtype=np.intc))


for ispin in range(nspin):
    spin_line = [int(d) for d in inpf.readline().split(' ')]
    outf.write_record(np.array(spin_line, dtype=np.intc))
    nmo, homo, lfomo, nelectron = spin_line
    
    evals_occs = [float(d) for d in inpf.readline().split(' ')]
    outf.write_record(np.array(evals_occs, dtype=np.float64))
    
    for imo in range(nmo):
        coefs = [float(d) for d in inpf.readline().split(' ')]
        outf.write_record(np.array(coefs, dtype=np.float64))
    
inpf.close()
outf.close()